# Chapter 12: Custom Models and Training with TensorFlow - Notebook Reproduksi Kode

Bab ini membahas bagaimana membuat komponen kustom di TensorFlow dan Keras,
serta menulis loop pelatihan kustom menggunakan API tingkat rendah TensorFlow.

Kita akan melihat:
- Fungsi Kerugian Kustom (Custom Loss Functions).
- Fungsi Aktivasi, Initializer, Regularizer, Constraint Kustom.
- Model Kustom (menggunakan subclassing keras.Model).
- Loop Pelatihan Kustom (Custom Training Loops) dengan tf.GradientTape.
- Layer Kustom.

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing # Untuk dataset California Housing

# Atur random seed untuk reproduksibilitas
np.random.seed(42)
tf.random.set_seed(42)

# Path untuk menyimpan gambar plot
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "custom_nn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

# Memuat Dataset Fashion MNIST
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full_fashion, y_train_full_fashion), (X_test_fashion, y_test_fashion) = fashion_mnist.load_data()

X_valid_fashion, X_train_fashion = X_train_full_fashion[:5000] / 255.0, X_train_full_fashion[5000:] / 255.0
y_valid_fashion, y_train_fashion = y_train_full_fashion[:5000], y_train_full_fashion[5000:]
X_test_fashion = X_test_fashion / 255.0

class_names_fashion = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                       "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

# Memuat Dataset California Housing (untuk demonstrasi regresi)
housing = fetch_california_housing()
X_train_full_housing, X_test_housing, y_train_full_housing, y_test_housing = train_test_split(
    housing.data, housing.target, random_state=42)
X_train_housing, X_valid_housing, y_train_housing, y_valid_housing = train_test_split(
    X_train_full_housing, y_train_full_housing, random_state=42)

scaler_housing = StandardScaler()
X_train_scaled_housing = scaler_housing.fit_transform(X_train_housing)
X_valid_scaled_housing = scaler_housing.transform(X_valid_housing)
X_test_scaled_housing = scaler_housing.transform(X_test_housing)

# Fungsi pembantu untuk plot kurva pembelajaran (opsional, karena tidak semua bagian akan di-plot)
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    # plt.gca().set_ylim(0, 1) # Jika metrik akurasi/loss
    plt.title("Kurva Pembelajaran")
    plt.xlabel("Epoch")
    plt.ylabel("Metrik")
    plt.show()

# --- 1. Fungsi Kerugian Kustom (Custom Loss Functions) ---
# Contoh: Huber Loss - kurang sensitif terhadap outlier dibandingkan MSE.

print("--- Fungsi Kerugian Kustom (Huber Loss) ---")
def huber_loss(y_true, y_pred, delta=1.0):
    error = tf.abs(y_true - y_pred)
    # Kondisi: jika error <= delta, gunakan 0.5 * error^2
    # Jika error > delta, gunakan delta * error - 0.5 * delta^2
    is_small_error = error < delta
    squared_loss = 0.5 * tf.square(error)
    linear_loss = delta * error - 0.5 * tf.square(delta)
    return tf.where(is_small_error, squared_loss, linear_loss)

# Membuat model sederhana untuk demonstrasi
model_huber_loss = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train_scaled_housing.shape[1:]),
    keras.layers.Dense(1)
])

# Mengkompilasi model dengan custom loss
# Penting: Saat menyimpan/memuat model dengan custom object, perlu custom_objects dict
print("\nMelatih model dengan Huber Loss kustom...")
model_huber_loss.compile(loss=huber_loss, optimizer="adam")
history_huber = model_huber_loss.fit(X_train_scaled_housing, y_train_housing, epochs=10,
                                     validation_data=(X_valid_scaled_housing, y_valid_housing), verbose=0) # verbose=0 agar tidak membanjiri output
print("Model dengan Huber Loss kustom berhasil dilatih.")
print(f"MSE pada set pengujian: {model_huber_loss.evaluate(X_test_scaled_housing, y_test_housing, verbose=0):.4f}")


# --- 2. Fungsi Aktivasi, Initializer, Regularizer, Constraint Kustom ---

# a. Fungsi Aktivasi Kustom (contoh: PReLU)
print("\n--- Fungsi Aktivasi Kustom (PReLU Sederhana) ---")
def custom_prelu(z, alpha=0.1): # Implementasi sederhana PReLU
    return tf.where(z < 0, alpha * z, z)

model_custom_activation = keras.models.Sequential([
    keras.layers.Dense(30, input_shape=X_train_scaled_housing.shape[1:]),
    keras.layers.Activation(custom_prelu), # Menggunakan fungsi aktivasi kustom
    keras.layers.Dense(1)
])
model_custom_activation.compile(loss="mse", optimizer="adam")
print("Melatih model dengan aktivasi PReLU kustom...")
model_custom_activation.fit(X_train_scaled_housing, y_train_housing, epochs=5, verbose=0)
print("Model dengan aktivasi kustom berhasil dilatih.")


# b. Initializer Kustom (contoh: all zeros)
print("\n--- Initializer Kustom (All Zeros) ---")
def zero_initializer(shape, dtype=tf.float32):
    return tf.zeros(shape, dtype=dtype)

model_custom_initializer = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=zero_initializer,
                        input_shape=X_train_scaled_housing.shape[1:]),
    keras.layers.Dense(1)
])
model_custom_initializer.compile(loss="mse", optimizer="adam")
print("Melatih model dengan initializer kustom...")
model_custom_initializer.fit(X_train_scaled_housing, y_train_housing, epochs=1, verbose=0) # Epoch singkat karena zero init mungkin tidak bagus
print("Model dengan initializer kustom berhasil dilatih.")


# c. Regularizer Kustom (contoh: L1 custom)
print("\n--- Regularizer Kustom (L1 Custom) ---")
# FIX: Mengubah fungsi biasa menjadi kelas yang mewarisi dari keras.regularizers.Regularizer
class CustomL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, strength):
        self.strength = strength

    def __call__(self, weight_matrix):
        return self.strength * tf.reduce_sum(tf.abs(weight_matrix))

    def get_config(self):
        return {"strength": self.strength}

model_custom_regularizer = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_regularizer=CustomL1Regularizer(0.01), # Menggunakan instance dari kelas kustom
                        input_shape=X_train_scaled_housing.shape[1:]),
    keras.layers.Dense(1)
])
model_custom_regularizer.compile(loss="mse", optimizer="adam")
print("Melatih model dengan regularizer L1 kustom...")
model_custom_regularizer.fit(X_train_scaled_housing, y_train_housing, epochs=5, verbose=0)
print("Model dengan regularizer kustom berhasil dilatih.")


# d. Constraint Kustom (contoh: Non-negative weights)
print("\n--- Constraint Kustom (Non-negative Weights) ---")
# FIX: Mengubah fungsi biasa menjadi kelas yang mewarisi dari keras.constraints.Constraint
class NonNegWeights(keras.constraints.Constraint):
    def __call__(self, weights):
        return tf.cast(tf.greater_equal(weights, 0.), tf.float32) * weights # Set negatif ke 0

    def get_config(self):
        return {} # Constraint sederhana tidak butuh konfigurasi khusus

model_custom_constraint = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_constraint=NonNegWeights(), # Menggunakan instance dari kelas kustom
                        input_shape=X_train_scaled_housing.shape[1:]),
    keras.layers.Dense(1)
])
model_custom_constraint.compile(loss="mse", optimizer="adam")
print("Melatih model dengan constraint kustom...")
model_custom_constraint.fit(X_train_scaled_housing, y_train_housing, epochs=5, verbose=0)
print("Model dengan constraint kustom berhasil dilatih.")


# --- 3. Model Kustom (menggunakan subclassing keras.Model) ---
# Contoh: Model dengan residual connections

print("\n--- Model Kustom (Subclassing keras.Model) ---")
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_neurons, activation, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(n_neurons, activation=activation, kernel_initializer="he_normal")
        self.hidden2 = keras.layers.Dense(n_neurons, activation=activation, kernel_initializer="he_normal")
        # Perhatikan bahwa residual block harus mengembalikan bentuk yang sama dengan inputnya
        # Jika input dan output dari hidden layers tidak sama, perlu proyeksi linier (1x1 conv/dense)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        Z = self.hidden2(Z)
        # Residual connection: pastikan inputs dan Z memiliki dimensi yang kompatibel
        # Jika dimensi berubah, tambahkan lapisan Dense atau konvolusi untuk menyamakan
        if inputs.shape[-1] != Z.shape[-1]: # Contoh penanganan jika dimensi tidak cocok
            inputs = keras.layers.Dense(Z.shape[-1], activation=None)(inputs) # Proyeksi linier
        return inputs + Z

class ResidualDNN(keras.Model):
    def __init__(self, n_hidden_blocks, n_neurons_per_block, output_neurons, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.initial_dense = keras.layers.Dense(n_neurons_per_block, activation=activation, kernel_initializer="he_normal")
        self.blocks = [ResidualBlock(n_neurons_per_block, activation, name=f"res_block_{i}")
                       for i in range(n_hidden_blocks)]
        self.output_layer = keras.layers.Dense(output_neurons)

    def call(self, inputs):
        x = self.initial_dense(inputs)
        for block in self.blocks:
            x = block(x)
        return self.output_layer(x)

# Membuat dan melatih model ResidualDNN
# Perhatikan bahwa input_shape perlu diberikan saat membuat instance untuk .build() atau pertama kali .call()
model_custom_subclass = ResidualDNN(n_hidden_blocks=3, n_neurons_per_block=50, output_neurons=1)

# Membangun model agar summary bisa muncul
model_custom_subclass.build(input_shape=(None, X_train_scaled_housing.shape[1])) # (None, num_features)

print("\nRingkasan Model Kustom (ResidualDNN):")
model_custom_subclass.summary()

model_custom_subclass.compile(loss="mse", optimizer="adam")
print("\nMelatih model kustom (subclassing)...")
history_custom_subclass = model_custom_subclass.fit(X_train_scaled_housing, y_train_housing, epochs=10,
                                                      validation_data=(X_valid_scaled_housing, y_valid_housing), verbose=0)
print("Model kustom (subclassing) berhasil dilatih.")


# --- 4. Loop Pelatihan Kustom (Custom Training Loops) ---
# Menggunakan tf.GradientTape untuk kontrol penuh.

print("\n--- Loop Pelatihan Kustom ---")

# a. Definisikan Model (sederhana)
keras.backend.clear_session() # Hapus graph sebelumnya
tf.random.set_seed(42)
np.random.seed(42)

l2_reg = keras.regularizers.l2(0.05)
model_custom_loop = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l2_reg,
                        input_shape=X_train_scaled_housing.shape[1:]),
    keras.layers.Dense(1, kernel_initializer="he_normal", kernel_regularizer=l2_reg)
])

# b. Definisikan Hyperparameter Pelatihan
optimizer_custom_loop = keras.optimizers.Nadam(learning_rate=0.001)
loss_fn_custom_loop = keras.losses.MeanSquaredError()

mean_loss = keras.metrics.Mean()
mean_val_loss = keras.metrics.Mean()

# c. Loop Pelatihan Manual
n_epochs_custom_loop = 10
batch_size_custom_loop = 32

# Mengatur dataset ke dalam tf.data.Dataset
train_set = tf.data.Dataset.from_tensor_slices((X_train_scaled_housing, y_train_housing)).shuffle(1000).batch(batch_size_custom_loop).prefetch(1)
valid_set = tf.data.Dataset.from_tensor_slices((X_valid_scaled_housing, y_valid_housing)).batch(batch_size_custom_loop).prefetch(1)

print("\nMemulai loop pelatihan kustom...")
for epoch in range(n_epochs_custom_loop):
    # Loop pelatihan
    for X_batch, y_batch in train_set:
        with tf.GradientTape() as tape:
            y_pred = model_custom_loop(X_batch, training=True) # training=True untuk layer seperti Dropout/BN
            main_loss = loss_fn_custom_loop(y_batch, y_pred)
            # Menambahkan loss regularisasi manual (jika ada)
            loss = tf.add_n([main_loss] + model_custom_loop.losses)

        # Hitung gradien
        gradients = tape.gradient(loss, model_custom_loop.trainable_variables)

        # Update bobot
        optimizer_custom_loop.apply_gradients(zip(gradients, model_custom_loop.trainable_variables))

        # Update metrik
        mean_loss(loss) # Pass loss total (termasuk regularisasi)

    # Loop validasi
    for X_batch_val, y_batch_val in valid_set:
        y_pred_val = model_custom_loop(X_batch_val)
        val_loss = loss_fn_custom_loop(y_batch_val, y_pred_val)
        # Tidak perlu menambahkan loss regularisasi untuk validasi, hanya main loss
        mean_val_loss(val_loss)

    print(f"Epoch {epoch + 1}/{n_epochs_custom_loop} - Loss: {mean_loss.result():.4f}, Val_loss: {mean_val_loss.result():.4f}")

    # Reset metrik untuk epoch berikutnya
    mean_loss.reset_state()
    mean_val_loss.reset_state()

print("Loop pelatihan kustom selesai.")

# --- 5. Layer Kustom ---
# Contoh: Sebuah layer kustom yang tidak hanya melakukan operasi Dense,
# tapi juga memiliki logika internal khusus.

print("\n--- Layer Kustom ---")
class DenseWithActivationAndBiasConstraint(keras.layers.Layer):
    def __init__(self, units, activation=None, bias_constraint=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        self.bias_constraint = keras.constraints.get(bias_constraint)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            name="kernel",
            shape=(input_shape[-1], self.units),
            initializer="he_normal",
            trainable=True
        )
        self.bias = self.add_weight(
            name="bias",
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            constraint=self.bias_constraint
        )
        super().build(input_shape) # Wajib panggil ini di akhir build()

    def call(self, inputs):
        Z = tf.matmul(inputs, self.kernel) + self.bias
        if self.activation is not None:
            Z = self.activation(Z)
        return Z

    def get_config(self):
        # Penting untuk menyimpan dan memulihkan layer kustom
        config = super().get_config()
        config.update({
            "units": self.units,
            "activation": keras.activations.serialize(self.activation),
            "bias_constraint": keras.constraints.serialize(self.bias_constraint),
        })
        return config

# Membuat dan melatih model dengan layer kustom
model_custom_layer = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    DenseWithActivationAndBiasConstraint(300, activation="relu", bias_constraint="NonNeg", name="my_custom_dense_layer"),
    DenseWithActivationAndBiasConstraint(100, activation="relu", name="my_custom_dense_layer_2"),
    keras.layers.Dense(10, activation="softmax")
])

model_custom_layer.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
print("\nMelatih model dengan layer kustom...")
history_custom_layer = model_custom_layer.fit(X_train_fashion, y_train_fashion, epochs=5, verbose=0)
print("Model dengan layer kustom berhasil dilatih.")

# Menyimpan dan memuat model dengan layer kustom (membutuhkan custom_objects)
model_custom_layer.save("my_custom_layer_model.h5")
print("\nModel dengan layer kustom disimpan. Mencoba memuat...")
try:
    loaded_custom_layer_model = keras.models.load_model(
        "my_custom_layer_model.h5",
        custom_objects={"DenseWithActivationAndBiasConstraint": DenseWithActivationAndBiasConstraint,
                        "NonNegWeights": NonNegWeights} # FIX: Tambahkan NonNegWeights ke custom_objects
    )
    print("Model dengan layer kustom berhasil dimuat.")
    # Verifikasi dengan summary
    loaded_custom_layer_model.summary()
except Exception as e:
    print(f"Gagal memuat model dengan layer kustom: {e}")


print("\n--- Selesai Reproduksi Kode Chapter 12 ---")


--- Fungsi Kerugian Kustom (Huber Loss) ---

Melatih model dengan Huber Loss kustom...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model dengan Huber Loss kustom berhasil dilatih.
MSE pada set pengujian: 0.1589

--- Fungsi Aktivasi Kustom (PReLU Sederhana) ---
Melatih model dengan aktivasi PReLU kustom...
Model dengan aktivasi kustom berhasil dilatih.

--- Initializer Kustom (All Zeros) ---
Melatih model dengan initializer kustom...
Model dengan initializer kustom berhasil dilatih.

--- Regularizer Kustom (L1 Custom) ---
Melatih model dengan regularizer L1 kustom...
Model dengan regularizer kustom berhasil dilatih.

--- Constraint Kustom (Non-negative Weights) ---
Melatih model dengan constraint kustom...
Model dengan constraint kustom berhasil dilatih.

--- Model Kustom (Subclassing keras.Model) ---

Ringkasan Model Kustom (ResidualDNN):


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'residual_dnn', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "residual_dnn"

...

 Total params: 266,612 (1.02 MB)

 Trainable params: 266,610 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)


--- Selesai Reproduksi Kode Chapter 12 ---
